Primero que nada vamos a importar pandas, una librería que se usa en el contexto de la manipulación de datos, en específico datos tabulares como lo pueden ser hojas de cálculo, base de datos o en mi caso que son archivos .CSV, esto ya que necesitamos unificar distintos tipos de archivos .CSV que vienen de diferentes fuentes

In [1]:
import pandas as pd

Ahora vamos a importar el df1 que es el principal, cabe recalcar que la columna de comuna, no fue extraída mediante scraping, esta fue mediante la descarga de un archivo Shapely y luego mediante librerías como, shapely, geopandas, matplotlib y folium se pudieron obtener los polígonos de las comunas a partir de las coordenadas, además esto es bastante conveniente tomando en cuenta que sirve para todo el territorio de Chile, no solo la región metropolitana

In [2]:
df1 = pd.read_csv("../../data/interim/propiedades_con_comuna.csv", encoding="utf-8", delimiter=",")
df1.head()

,precio,banos,dormitorios,superficie_total,superficie_construida,estacionamiento,latitud,longitud,comuna
0,322633576.0,3.0,5.0,404.0,275.0,1.0,-33.345468,-70.678398,Huechuraba
1,124089837.0,3.0,3.0,102.0,27.0,1.0,-33.563160,-70.776609,Maipú
2,157180460.0,2.0,3.0,190.0,127.0,1.0,-33.563025,-70.559425,Puente Alto
3,133544301.0,2.0,5.0,146.0,61.0,1.0,-33.641678,-70.689872,San Bernardo
4,117786861.0,2.0,3.0,145.0,82.0,1.0,-33.613778,-70.887450,Peñaflor


En el df1 aunque existan valores nulos o NaN, no los imputare, esto debido a que intenté usar KNN para imputar valores NaN y el rendimiento de los modelos disminuyó, y es que a veces es mejor no imputar datos y simplemente eliminarlos, cosa que en el caso del df2, no sucedió

In [3]:
df1.isna().sum()

precio                   11
banos                    47
dormitorios              38
superficie_total         59
superficie_construida    83
estacionamiento          30
latitud                  13
longitud                 13
comuna                   20
dtype: int64

Por el lado del df2, este fue extraído mediante scraping de datos de la mapoteca de la Biblioteca del Congreso Nacional de Chile, el cual tiene datos de todas las comunas de Chile, incluyendo datos, demográficos, sociales, salud, económicos, seguridad, educacionales y electorales, esto debido a que se extrajeron datos de 52 comunas, y cada comuna tenía 29 variables, por lo cual mediante scraping se hace la tarea mucho más amena y eficiente

In [ ]:
df2 = pd.read_csv("../../data/external/scraped/datos_comunas_extraidos.csv", encoding="utf-8", delimiter=",")
df2.head()

,Comuna,Población 2024,Índice Masculinidad 2024,Grupo_Etario_0_a_14_2024_(%),Grupo_Etario_15_a_29_2024_(%),Grupo_Etario_30_a_44_2024_(%),Grupo_Etario_45_a_64_2024_(%),Grupo_Etario_65_o_mas_2024_(%),IDD 2024,IAM 2024,...,Trabajadores_Enseñanza_2023,Trabajadores_Salud_2023,Trabajadores_Artísticas_Entretenimiento_2023,Trabajadores_Otras_Actividades_2023,Trabajadores_Hogares_Empleadores_2023,Trabajadores_Organizaciones_Extraterritoriales_2023,Trabajadores_Sin_Información_2023,Disponibilidad_Presupuestaria_por_Habitante_2023_(M$),Delitos Mayor Connotación Social 2023 (c/100.000 hab),Violencia Intrafamiliar 2023 (c/100.000 hab)
0,Alhué,7768,98.9,20.7,20.0,23.3,23.0,13.0,51.0,62.8,...,0,5,0,11,0,0,0,2724,1947.6,929.2
1,Buin,116969,93.6,21.3,19.6,26.5,22.2,10.4,46.4,48.7,...,1614,1788,602,433,0,0,10,341,1982.5,459.4
2,Calera de Tango,25491,97.9,18.0,21.1,20.7,26.7,13.5,46.0,74.8,...,492,70,10,778,0,0,4,487,2044.1,548.2
3,Cerrillos,85041,94.0,18.7,21.7,22.9,23.7,13.0,46.3,69.7,...,947,1179,39,645,0,0,31,335,3434.2,682.0
4,Cerro Navia,127250,96.0,17.6,21.6,20.6,25.4,14.7,47.7,83.8,...,1312,4044,25,1210,0,0,1,286,3053.7,675.3


In [5]:
guiones_total = (df2 == "-").sum().sum()
guiones_mortalidad = (df2['Mortalidad Infantil 2022 (c/1.000 nac.vivos)'] == "-").sum()

print(f'Total de NaN: {guiones_total}\n'
      f'NaN en mortalidad infantil: {guiones_mortalidad}')

Total de NaN: 4
NaN en mortalidad infantil: 4


Los modelos de machine learning son muy útiles cuando se trata de imputación de datos, y es que hay que aprovechar que disponemos de una amplia gama de variables que podemos usar para apalancarnos y usar modelos como random forest y no perder datos valiosos dentro de nuestro dataset, hay muchas herramientas que se pueden usar para imputar datos, como media, mediana, o moda, pero también se pueden usar métodos estadísticos, generalmente se usa k-Nearest Neighbors para la imputación de datos, pero yo creo que para este caso con 4 comunas sin datos para la columna de mortalidad infantil, debido a la alta cantidad de variables random forest es mejor método para la imputación para este caso

In [6]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

target = 'Mortalidad Infantil 2022 (c/1.000 nac.vivos)'
df2[target] = df2[target].replace('-', np.nan)
df2[target] = pd.to_numeric(df2[target], errors='coerce')
complete_data = df2.dropna(subset=[target])

predictores = [
    'Pobreza por ingresos 2022 (%)',
    'Pobreza multidimensional 2022 (%)',
    'Mortalidad General 2022 (c/1.000 hab)',
    'Carentes servicios básicos 2025 (%)',
    'Hogares hacinados 2025 (%)',
    'Natalidad 2022',
    'Fecundidad 2022',
    'Cantidad de: Centro de Salud Familiar (CESFAM) 2025',
    'Cantidad de: Hospital 2025',
    'Grupo_Etario_0_a_14_2024_(%)',
    'Disponibilidad_Presupuestaria_por_Habitante_2023_(M$)',
    'SIMCE 4to Básico Lectura 2022'
]


In [7]:
X_train = complete_data[predictores]
y_train = complete_data[target]

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

missing_mask = df2[target].isnull()
X_missing = df2.loc[missing_mask, predictores]
predicted_values = model.predict(X_missing)

df2.loc[missing_mask, target] = np.round(predicted_values, 1)

Ahora lo que vamos a hacer es incorporar más variables relacionadas a las características de las casas en cada comuna según la base de datos de CASEN 2022 para luego unificar toda esta

In [ ]:
df3 = pd.read_csv("../../data/interim/caracteristicas_vivienda_rm_casen2022.csv", encoding="utf-8", delimiter=",")
df3 = df3.drop('codigo_comuna', axis=1)
df3 = df3.rename(columns={'nombre_comuna': 'comuna'})
df3.head()

,comuna,n_registros,pct_casa,pct_departamento,pct_casa_depto,pct_vivienda_precaria,pct_paredes_solidas,pct_techo_solido,pct_piso_bueno,pct_agua_red_publica,pct_wc_bueno,pct_cocina_moderna,pct_propia_total,pct_arrendada,ingreso_promedio,ingreso_mediana,indice_calidad_materiales,indice_servicios_basicos,indice_calidad_vivienda_general
0,Santiago,1793,0.78,2.79,3.57,75.46,87.62,96.04,95.37,68.93,73.73,94.70,18.96,0.11,791945.92,600000.0,93.01,79.12,58.57
1,Cerrillos,401,28.93,40.40,69.33,18.45,83.79,95.51,95.26,71.07,36.91,87.28,51.37,3.74,528548.10,450000.0,91.52,65.09,75.31
2,Cerro Navia,555,39.28,26.13,65.41,7.03,78.02,97.48,89.01,68.65,11.89,94.41,61.98,1.98,446483.01,400000.0,88.17,58.32,70.63
3,Conchalí,560,15.89,56.96,72.86,17.32,78.75,84.11,93.04,41.61,16.25,76.79,46.96,0.36,456803.06,400000.0,85.30,44.88,67.68
4,El Bosque,810,19.26,47.28,66.54,15.80,85.43,96.30,93.83,76.79,14.69,94.07,57.78,0.99,395604.34,350000.0,91.85,61.85,73.42


Luego de tener los 2 datasets, lo que hice fue hacerles un "merge" que consiste en unir estos 2 datasets, utilizando la columna ['comuna'] como una especie de llave o "key" que alimenta con más variables cada fila

In [9]:
df1['comuna'] = df1['comuna'].str.strip().str.lower()
df2['Comuna'] = df2['Comuna'].str.strip().str.lower()
df3['comuna'] = df3['comuna'].str.strip().str.lower()

df2.columns = df2.columns.str.lower()

df = pd.merge(df1, df2, on='comuna', how='left')
df = pd.merge(df, df3, on='comuna', how='left')

df.head()

,precio,banos,dormitorios,superficie_total,superficie_construida,estacionamiento,latitud,longitud,comuna,población 2024,...,pct_agua_red_publica,pct_wc_bueno,pct_cocina_moderna,pct_propia_total,pct_arrendada,ingreso_promedio,ingreso_mediana,indice_calidad_materiales,indice_servicios_basicos,indice_calidad_vivienda_general
0,322633576.0,3.0,5.0,404.0,275.0,1.0,-33.345468,-70.678398,huechuraba,101808.0,...,61.78,32.43,89.86,64.31,0.00,720051.66,402500.0,86.47,61.35,74.64
1,124089837.0,3.0,3.0,102.0,27.0,1.0,-33.563160,-70.776609,maipú,503635.0,...,76.69,38.93,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23
2,157180460.0,2.0,3.0,190.0,127.0,1.0,-33.563025,-70.559425,puente alto,568086.0,...,72.70,34.05,90.62,69.71,0.18,538232.86,400000.0,93.75,65.79,78.18
3,133544301.0,2.0,5.0,146.0,61.0,1.0,-33.641678,-70.689872,san bernardo,306371.0,...,70.58,26.52,90.21,58.31,1.16,488733.18,400000.0,89.94,62.44,76.41
4,117786861.0,2.0,3.0,145.0,82.0,1.0,-33.613778,-70.887450,peñaflor,94402.0,...,72.05,25.70,89.12,65.85,0.19,591753.35,400000.0,84.18,62.29,77.28


Luego lo que vamos a hacer es crear un archivo CSV a partir del df, y vamos a importar las coordenadas de todos los hospitales y estaciones de metro, estos al igual que el caso de los datos de las comunas, son extraídos a partir de archivos Shapely mediante python y respectivas librerías ya mencionadas, estas coordenadas de hospitales y metros son usadas para luego calcular la distancia lineal entre las casas y el hospital y metro más cercano

In [ ]:
df_hospitales = pd.read_csv("../../data/interim/coordenadas_hospitales_publicos_rm.csv", encoding="utf-8", delimiter=",")
df_metro = pd.read_csv("../../data/interim/coordenadas_estaciones_metro.csv", encoding="utf-8", delimiter=",")

In [11]:
df.head()

,precio,banos,dormitorios,superficie_total,superficie_construida,estacionamiento,latitud,longitud,comuna,población 2024,...,pct_agua_red_publica,pct_wc_bueno,pct_cocina_moderna,pct_propia_total,pct_arrendada,ingreso_promedio,ingreso_mediana,indice_calidad_materiales,indice_servicios_basicos,indice_calidad_vivienda_general
0,322633576.0,3.0,5.0,404.0,275.0,1.0,-33.345468,-70.678398,huechuraba,101808.0,...,61.78,32.43,89.86,64.31,0.00,720051.66,402500.0,86.47,61.35,74.64
1,124089837.0,3.0,3.0,102.0,27.0,1.0,-33.563160,-70.776609,maipú,503635.0,...,76.69,38.93,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23
2,157180460.0,2.0,3.0,190.0,127.0,1.0,-33.563025,-70.559425,puente alto,568086.0,...,72.70,34.05,90.62,69.71,0.18,538232.86,400000.0,93.75,65.79,78.18
3,133544301.0,2.0,5.0,146.0,61.0,1.0,-33.641678,-70.689872,san bernardo,306371.0,...,70.58,26.52,90.21,58.31,1.16,488733.18,400000.0,89.94,62.44,76.41
4,117786861.0,2.0,3.0,145.0,82.0,1.0,-33.613778,-70.887450,peñaflor,94402.0,...,72.05,25.70,89.12,65.85,0.19,591753.35,400000.0,84.18,62.29,77.28


La siguiente celda lo que hace es calcular la distancia entre la coordenada de la casa, y la coordenada del hospital más cercano y luego crear una columna llamada 'distancia_hospital' y agregarla al data frame

In [12]:
prop_coords = df[['latitud', 'longitud']].to_numpy()
hospital_coords = df_hospitales[['latitud hospital', 'longitud hospital']].to_numpy()

prop_coords_rad = np.radians(prop_coords)
hospital_coords_rad = np.radians(hospital_coords)

def haversine_vectorized(coord1, coord2):
    R = 6371000 

    lat1 = coord1[:, 0][:, np.newaxis]
    lon1 = coord1[:, 1][:, np.newaxis]
    lat2 = coord2[:, 0]
    lon2 = coord2[:, 1]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c  

dist_matrix = haversine_vectorized(prop_coords_rad, hospital_coords_rad)
df['distancia_hospital'] = dist_matrix.min(axis=1)
df


,precio,banos,dormitorios,superficie_total,superficie_construida,estacionamiento,latitud,longitud,comuna,población 2024,...,pct_wc_bueno,pct_cocina_moderna,pct_propia_total,pct_arrendada,ingreso_promedio,ingreso_mediana,indice_calidad_materiales,indice_servicios_basicos,indice_calidad_vivienda_general,distancia_hospital
0,322633576.0,3.0,5.0,404.0,275.0,1.0,-33.345468,-70.678398,huechuraba,101808.0,...,32.43,89.86,64.31,0.00,720051.66,402500.0,86.47,61.35,74.64,8064.787023
1,124089837.0,3.0,3.0,102.0,27.0,1.0,-33.563160,-70.776609,maipú,503635.0,...,38.93,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23,6113.888686
2,157180460.0,2.0,3.0,190.0,127.0,1.0,-33.563025,-70.559425,puente alto,568086.0,...,34.05,90.62,69.71,0.18,538232.86,400000.0,93.75,65.79,78.18,845.094730
3,133544301.0,2.0,5.0,146.0,61.0,1.0,-33.641678,-70.689872,san bernardo,306371.0,...,26.52,90.21,58.31,1.16,488733.18,400000.0,89.94,62.44,76.41,5475.616211
4,117786861.0,2.0,3.0,145.0,82.0,1.0,-33.613778,-70.887450,peñaflor,94402.0,...,25.70,89.12,65.85,0.19,591753.35,400000.0,84.18,62.29,77.28,1600.028166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,742425000.0,5.0,5.0,370.0,230.0,0.0,-33.421135,-70.553193,las condes,296134.0,...,88.15,98.33,56.67,0.49,1806493.23,1050000.0,97.94,92.51,77.68,4638.577959
5966,69709800.0,1.0,2.0,88.0,70.0,0.0,-33.513907,-70.783665,maipú,503635.0,...,38.93,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23,1078.534716
5967,75000000.0,2.0,4.0,100.0,80.0,0.0,-33.416357,-70.765798,cerro navia,127250.0,...,11.89,94.41,61.98,1.98,446483.01,400000.0,88.17,58.32,70.63,2381.181842
5968,859650000.0,5.0,6.0,1000.0,320.0,0.0,-34.147102,-71.462988,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55220.407889


La siguiente celda lo que hace es lo mismo que la celda anterior, pero la diferencia es que aquí nosotros calculamos la distancia al punto de coordenadas de estación de metro más cercano y creamos la columna llamada 'distancia_metro' que luego es agregada al dataframe, hay que tomar en cuenta que para poder saber la distancia lineal en metros, se requiere del radio de la tierra en metros definido en la variable R que aparece en la celda de abajo

In [13]:
prop_coords = df[['latitud', 'longitud']].to_numpy()
metro_coords = df_metro[['latitud_estacion_metro', 'longitud_estacion_metro']].to_numpy()

prop_coords_rad = np.radians(prop_coords)
metro_coords_rad = np.radians(metro_coords)

def haversine_vectorized(coord1, coord2):
    R = 6371000  

    lat1 = coord1[:, 0][:, np.newaxis]
    lon1 = coord1[:, 1][:, np.newaxis]
    lat2 = coord2[:, 0]
    lon2 = coord2[:, 1]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

dist_matrix = haversine_vectorized(prop_coords_rad, metro_coords_rad)

df['distancia_metro'] = dist_matrix.min(axis=1)

df


,precio,banos,dormitorios,superficie_total,superficie_construida,estacionamiento,latitud,longitud,comuna,población 2024,...,pct_cocina_moderna,pct_propia_total,pct_arrendada,ingreso_promedio,ingreso_mediana,indice_calidad_materiales,indice_servicios_basicos,indice_calidad_vivienda_general,distancia_hospital,distancia_metro
0,322633576.0,3.0,5.0,404.0,275.0,1.0,-33.345468,-70.678398,huechuraba,101808.0,...,89.86,64.31,0.00,720051.66,402500.0,86.47,61.35,74.64,8064.787023,2554.229150
1,124089837.0,3.0,3.0,102.0,27.0,1.0,-33.563160,-70.776609,maipú,503635.0,...,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23,6113.888686,6127.575990
2,157180460.0,2.0,3.0,190.0,127.0,1.0,-33.563025,-70.559425,puente alto,568086.0,...,90.62,69.71,0.18,538232.86,400000.0,93.75,65.79,78.18,845.094730,2324.734493
3,133544301.0,2.0,5.0,146.0,61.0,1.0,-33.641678,-70.689872,san bernardo,306371.0,...,90.21,58.31,1.16,488733.18,400000.0,89.94,62.44,76.41,5475.616211,11151.846959
4,117786861.0,2.0,3.0,145.0,82.0,1.0,-33.613778,-70.887450,peñaflor,94402.0,...,89.12,65.85,0.19,591753.35,400000.0,84.18,62.29,77.28,1600.028166,16662.321199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,742425000.0,5.0,5.0,370.0,230.0,0.0,-33.421135,-70.553193,las condes,296134.0,...,98.33,56.67,0.49,1806493.23,1050000.0,97.94,92.51,77.68,4638.577959,1526.624955
5966,69709800.0,1.0,2.0,88.0,70.0,0.0,-33.513907,-70.783665,maipú,503635.0,...,92.76,70.40,0.80,601743.03,450000.0,94.91,69.46,78.23,1078.534716,2485.915755
5967,75000000.0,2.0,4.0,100.0,80.0,0.0,-33.416357,-70.765798,cerro navia,127250.0,...,94.41,61.98,1.98,446483.01,400000.0,88.17,58.32,70.63,2381.181842,3924.559734
5968,859650000.0,5.0,6.0,1000.0,320.0,0.0,-34.147102,-71.462988,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55220.407889,96236.455245


In [ ]:
df.to_csv("../../data/interim/datos_variables_completas.csv", index=False)

In [16]:
print(df.columns.tolist())

['precio', 'banos', 'dormitorios', 'superficie_total', 'superficie_construida', 'estacionamiento', 'latitud', 'longitud', 'comuna', 'población 2024', 'índice masculinidad 2024', 'grupo_etario_0_a_14_2024_(%)', 'grupo_etario_15_a_29_2024_(%)', 'grupo_etario_30_a_44_2024_(%)', 'grupo_etario_45_a_64_2024_(%)', 'grupo_etario_65_o_mas_2024_(%)', 'idd 2024', 'iam 2024', 'pobreza por ingresos 2022 (%)', 'pobreza multidimensional 2022 (%)', 'pueblos indígenas 2025 (%)', 'extranjeros 2025 (%)', 'carentes servicios básicos 2025 (%)', 'hogares hacinados 2025 (%)', 'cantidad de: atención remota 2025', 'cantidad de: centro comunitario de salud familiar (cecosf) 2025', 'cantidad de: centro comunitario de salud mental  (cosam) 2025', 'cantidad de: centro corporación para la nutrición infantil (conin) 2025', 'cantidad de: centro de apoyo comunitario para personas con demencia 2025', 'cantidad de: centro diagnóstico terapéutico y tratamiento privado (cdt) 2025', 'cantidad de: centro de diálisis 2025', 